In [7]:
import pandas as pd
import numpy as np
import cv2
import os
from skimage.feature import local_binary_pattern
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split

# Fungsi untuk menghitung LBP pada satu gambar
def calculate_lbp_image(img):
    lbp_image = local_binary_pattern(img, 8, 1, method='uniform')
    return lbp_image

# Fungsi untuk menyiapkan dataset dari gambar
def prepare_dataset(data_dir):
    labels = []
    lbp_features = []
    emotions = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
    
    for emotion in emotions:
        emotion_dir = os.path.join(data_dir, emotion)
        for file_name in os.listdir(emotion_dir):
            file_path = os.path.join(emotion_dir, file_name)
            img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
            lbp_image = calculate_lbp_image(img)
            lbp_features.append(lbp_image)
            labels.append(emotion)
    
    # Konversi list ke array
    lbp_features = np.array(lbp_features, dtype='float32').reshape(-1, 48, 48, 1)
    # Konversi label emosi menjadi angka dan kemudian ke one-hot encoding
    encoder = LabelEncoder()
    labels_encoded = encoder.fit_transform(labels)
    labels_one_hot = tf.keras.utils.to_categorical(labels_encoded, num_classes=7)
    
    return lbp_features, labels_one_hot

# Tentukan path ke direktori 'train' dari dataset yang diekstrak
train_dir = 'Dataset\train'
lbp_features, labels_one_hot = prepare_dataset(train_dir)

# Bagi data menjadi set pelatihan dan validasi
X_train, X_val, y_train, y_val = train_test_split(lbp_features, labels_one_hot, test_size=0.2, random_state=42)

# Buat model Sequential
model = Sequential([
    Conv2D(64, (3, 3), input_shape=(48, 48, 1), activation='relu', padding='same'),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    Flatten(),
    Dense(4096, activation='relu'),
    Dropout(0.5),
    Dense(4096, activation='relu'),
    Dropout(0.5),
    Dense(7, activation='softmax')  # 7 kelas emosi
])

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Latih model dengan data LBP
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=64)

# Simpan model
model.save('lbp_emotion_recognition_model.h5')


OSError: [WinError 123] The filename, directory name, or volume label syntax is incorrect: 'Dataset\train\\angry'